In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
from sqlalchemy import create_engine, inspect
from utils.database_utils import *
from utils.config_utils import *
from constants import *

db_config = get_db_config(read_config(CONFIG_FILE_PATH))

engine = connect(db_config)

inspector = inspect(engine)

table_names = inspector.get_table_names()
table_names

Connection to PostgreSQL successful


['truck_schedule_table',
 'routes_table',
 'routes_weather',
 'trucks_table',
 'traffic_table',
 'drivers_table',
 'city_weather']

In [ ]:
import hopsworks
import pandas as pd

project = hopsworks.login(api_key_value = 'HIW4D6r4iHGmgSqT.W5LdkqjulcZ85kFXhePJdfJb2sxxZigzNOtweKlAXTLiou7QVkWPBzc9gfRXMjYC')
fs = project.get_feature_store()

feature_groups = [table + '_fg' for table in table_names]

feature_dataframes = {}

for fg_name in feature_groups:
    fg = fs.get_feature_group(fg_name, version=2) 
    
    query = fg.select_all()
    df=query.read(read_options={"use_hive": True})
    # df = fg.read(read_options={"use_hive": True})
    
    feature_dataframes[fg_name] = df

print(feature_dataframes)

Connected. Call `.close()` to terminate connection gracefully.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1022104
Connected. Call `.close()` to terminate connection gracefully.
2024-10-16 23:46:11,913 WARNING: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated

Finished: Reading data from Hopsworks, using Hive (18.42s) 
2024-10-16 23:46:31,846 WARNING: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated

Finished: Reading data from Hopsworks, using Hive (1.10s) 
2024-10-16 23:46:34,356 WARNING: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated

Finished: Reading data from Hopsworks, using Hive (34.55s) 
2024-10-16 23:47:10,030 WARNING: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated

Finished: Reading data from Hopsworks, using Hive (1.09s) 
2024-10-16 23:47:12,548 WARNING: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated

Finished: Reading data from Hopsworks, using Hive (133.95s) 
2024-10-16 23:49:28,535 WARNING: DeprecationWarning: ssl.PROTOCOL_TLS 

In [4]:
for key, df in feature_dataframes.items():
    if 'event_time' in df.columns:
        df.drop(columns=['event_time'], inplace=True)

In [5]:
city_weather_df = feature_dataframes['city_weather_fg']
drivers_df  = feature_dataframes['drivers_table_fg']
routes_df  = feature_dataframes['routes_table_fg']
routes_weather_df  = feature_dataframes['routes_weather_fg']
traffic_df  = feature_dataframes['traffic_table_fg']
truck_schedule_df  = feature_dataframes['truck_schedule_table_fg']
trucks_df  = feature_dataframes['trucks_table_fg']

In [6]:
city_weather_df

,id,city_id,date,hour,temp,wind_speed,description,precip,humidity,visibility,pressure,chanceofrain,chanceoffog,chanceofsnow,chanceofthunder
0,31378,C-9bbf5d8a,2019-01-21,900,77,6,Sunny,0.0,25,6,1020,0,0,0,0
1,23586,C-f8f01604,2019-01-17,1700,77,10,Sunny,0.0,12,6,1013,0,0,0,0
2,35280,C-03bb3e48,2019-02-14,2300,-6,11,Partly cloudy,0.0,90,6,1024,0,0,0,0
3,19059,C-3dbd8b2e,2019-01-13,200,43,8,Clear,0.0,70,6,1022,0,0,0,0
4,28860,C-b25a09de,2019-01-08,1100,55,7,Patchy rain possible,0.0,71,6,1011,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52071,46921,C-9bbf5d8a,2019-01-25,0,32,6,Clear,0.0,69,6,1018,0,0,0,0
52072,19439,C-3dbd8b2e,2019-01-28,2200,39,9,Clear,0.0,59,6,1022,0,0,0,0
52073,9078,C-639c5e36,2019-01-11,500,30,4,Patchy heavy snow,0.0,95,4,1028,0,0,0,0
52074,31159,C-9bbf5d8a,2019-01-12,600,77,6,Sunny,0.0,54,6,1009,0,0,0,0


# Preparation

## Drop duplicates

In [7]:
city_weather_df = city_weather_df.drop_duplicates(subset=['city_id', 'date', 'hour'], inplace=False)

routes_weather_df = routes_weather_df.drop_duplicates(subset=['route_id', 'date'], inplace=False)

trucks_df = trucks_df.drop_duplicates(subset=['truck_id'], inplace=False)

drivers_df = drivers_df.drop_duplicates(subset=['driver_id'], inplace=False)

routes_df = routes_df.drop_duplicates(subset=['route_id', 'destination_id', 'origin_id'], inplace=False)

truck_schedule_df = truck_schedule_df.drop_duplicates(subset=['truck_id', 'route_id', 'departure_date'], inplace=False)

traffic_df = traffic_df.drop_duplicates(subset=['route_id', 'date', 'hour'])

In [14]:
traffic_df

,id,route_id,date,hour,no_of_vehicles,accident,time_period
0,1189060,R-c8784955,2019-01-03,300,646.0,0,Early Morning
1,2350794,R-5cc73966,2019-01-08,800,664.0,0,Morning
2,1329575,R-23a9a043,2019-01-15,2200,2092.0,0,Night
3,340604,R-37c036aa,2019-01-24,1900,2295.0,0,Evening
4,2244644,R-a5925afd,2019-01-01,1000,2449.0,0,Late Morning
...,...,...,...,...,...,...,...
2597908,151033,R-d84c7160,2019-02-07,0,664.0,0,Late Night
2597909,874977,R-72cc6e25,2019-01-26,800,2817.0,0,Morning
2597910,2286416,R-b80fb464,2019-02-08,2200,2475.0,0,Night
2597911,2286145,R-b80fb464,2019-01-28,1500,2119.0,0,Late Afternoon


## Drop unnecesary columns

In [ ]:
city_weather_df = city_weather_df.drop(columns=['chanceofrain','chanceoffog','chanceofsnow','chanceofthunder'])

routes_weather_df = routes_weather_df.drop(columns=['chanceofrain','chanceoffog','chanceofsnow','chanceofthunder'])

## Change to date_time column

In [16]:
city_weather_df['date_time'] = city_weather_df['date'] + pd.to_timedelta(city_weather_df['hour'] // 100, unit='h')

In [17]:
city_weather_df = city_weather_df.drop(columns=['date','hour'])
city_weather_df.insert(1, 'date_time', city_weather_df.pop('date_time'))

In [18]:
traffic_df['date_time'] = traffic_df['date'] + pd.to_timedelta(traffic_df['hour'] // 100, unit='h')

2024-10-16 23:56:01,489 WARNING: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [19]:
traffic_df = traffic_df.drop(columns=['date','hour'])
traffic_df.insert(1, 'date_time', traffic_df.pop('date_time'))

In [20]:
traffic_df

,id,date_time,route_id,no_of_vehicles,accident,time_period
0,1189060,2019-01-03 03:00:00,R-c8784955,646.0,0,Early Morning
1,2350794,2019-01-08 08:00:00,R-5cc73966,664.0,0,Morning
2,1329575,2019-01-15 22:00:00,R-23a9a043,2092.0,0,Night
3,340604,2019-01-24 19:00:00,R-37c036aa,2295.0,0,Evening
4,2244644,2019-01-01 10:00:00,R-a5925afd,2449.0,0,Late Morning
...,...,...,...,...,...,...
2597908,151033,2019-02-07 00:00:00,R-d84c7160,664.0,0,Late Night
2597909,874977,2019-01-26 08:00:00,R-72cc6e25,2817.0,0,Morning
2597910,2286416,2019-02-08 22:00:00,R-b80fb464,2475.0,0,Night
2597911,2286145,2019-01-28 15:00:00,R-b80fb464,2119.0,0,Late Afternoon


# Feature Engineering

## Truck schedule and routes weather merge table

In [21]:
truck_schedule_df_fe = truck_schedule_df.copy()

In [22]:
truck_schedule_df_fe['departure_date'] = truck_schedule_df_fe['departure_date'].dt.floor('6H') 
truck_schedule_df_fe['estimated_arrival'] = truck_schedule_df_fe['estimated_arrival'].dt.ceil('6H') 
truck_schedule_df_fe

,id,truck_id,route_id,departure_date,estimated_arrival,delay
0,10626,14433691,R-8293c431,2019-02-10 06:00:00,2019-02-12 00:00:00,1
1,7780,21784492,R-2e17fb9b,2019-02-06 06:00:00,2019-02-07 06:00:00,0
2,3953,21680317,R-e4c16974,2019-01-19 06:00:00,2019-01-19 12:00:00,0
3,10264,52804295,R-ed86380e,2019-01-31 06:00:00,2019-02-02 00:00:00,1
4,10945,31870298,R-f5caf40b,2019-01-09 06:00:00,2019-01-11 00:00:00,0
...,...,...,...,...,...,...
12303,10077,11244695,R-b3146c56,2019-01-26 06:00:00,2019-01-27 12:00:00,0
12304,1416,64894212,R-91da63c5,2019-01-07 06:00:00,2019-01-08 06:00:00,0
12305,12265,37803062,R-9db6873e,2019-02-06 06:00:00,2019-02-13 18:00:00,0
12306,525,33014292,R-082bc641,2019-01-01 06:00:00,2019-01-01 18:00:00,0


In [ ]:

truck_schedule_df_fe['date'] = truck_schedule_df_fe.apply(
    lambda row: pd.date_range(start=row['departure_date'], 
                              end=row['estimated_arrival'], 
                              freq='6H'), axis=1)

truck_schedule_data_exploded = truck_schedule_df_fe.explode('date')
truck_schedule_data_exploded

,id,truck_id,route_id,departure_date,estimated_arrival,delay,date
0,10626,14433691,R-8293c431,2019-02-10 06:00:00,2019-02-12 00:00:00,1,2019-02-10 06:00:00
0,10626,14433691,R-8293c431,2019-02-10 06:00:00,2019-02-12 00:00:00,1,2019-02-10 12:00:00
0,10626,14433691,R-8293c431,2019-02-10 06:00:00,2019-02-12 00:00:00,1,2019-02-10 18:00:00
0,10626,14433691,R-8293c431,2019-02-10 06:00:00,2019-02-12 00:00:00,1,2019-02-11 00:00:00
0,10626,14433691,R-8293c431,2019-02-10 06:00:00,2019-02-12 00:00:00,1,2019-02-11 06:00:00
...,...,...,...,...,...,...,...
12306,525,33014292,R-082bc641,2019-01-01 06:00:00,2019-01-01 18:00:00,0,2019-01-01 18:00:00
12307,2886,15242565,R-038f3a7c,2019-01-13 06:00:00,2019-01-14 00:00:00,0,2019-01-13 06:00:00
12307,2886,15242565,R-038f3a7c,2019-01-13 06:00:00,2019-01-14 00:00:00,0,2019-01-13 12:00:00
12307,2886,15242565,R-038f3a7c,2019-01-13 06:00:00,2019-01-14 00:00:00,0,2019-01-13 18:00:00


In [ ]:
merged_df = pd.merge(truck_schedule_data_exploded, routes_weather_df, 
                     how='left', 
                     on=['route_id', 'date'])

In [25]:
routes_weather_df[routes_weather_df['route_id']=='R-266863d8'].sort_values(by = 'date')

,id,route_id,date,temp,wind_speed,description,precip,humidity,visibility,pressure
52926,71439,R-266863d8,2019-01-02 06:00:00,77,6,Sunny,0.0,14,6,1017
184392,71440,R-266863d8,2019-01-03 00:00:00,73,6,Clear,0.0,14,6,1017
200979,71441,R-266863d8,2019-01-03 06:00:00,73,7,Clear,0.0,14,6,1017
377097,293237,R-266863d8,2019-01-03 12:00:00,82,8,Clear,0.0,14,6,1010
126550,293238,R-266863d8,2019-01-03 18:00:00,81,8,Clear,0.0,15,6,1010
...,...,...,...,...,...,...,...,...,...,...
37009,293319,R-266863d8,2019-02-13 12:00:00,77,9,Sunny,0.0,19,6,1011
13340,293320,R-266863d8,2019-02-13 18:00:00,84,10,Sunny,0.0,16,6,1011
97507,71524,R-266863d8,2019-02-14 00:00:00,68,6,Sunny,0.0,22,6,1017
264667,71525,R-266863d8,2019-02-14 06:00:00,72,6,Sunny,0.0,18,6,1017


In [26]:
merged_df[merged_df['id_x']==347]

,id_x,truck_id,route_id,departure_date,estimated_arrival,delay,date,id_y,temp,wind_speed,description,precip,humidity,visibility,pressure
33877,347,21093078,R-266863d8,2019-01-01 06:00:00,2019-01-01 12:00:00,0,2019-01-01 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
33878,347,21093078,R-266863d8,2019-01-01 06:00:00,2019-01-01 12:00:00,0,2019-01-01 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [27]:
merged_df

,id_x,truck_id,route_id,departure_date,estimated_arrival,delay,date,id_y,temp,wind_speed,description,precip,humidity,visibility,pressure
0,10626,14433691,R-8293c431,2019-02-10 06:00:00,2019-02-12 00:00:00,1,2019-02-10 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10626,14433691,R-8293c431,2019-02-10 06:00:00,2019-02-12 00:00:00,1,2019-02-10 12:00:00,219513.0,73.0,11.0,Sunny,0.0,67.0,6.0,1017.0
2,10626,14433691,R-8293c431,2019-02-10 06:00:00,2019-02-12 00:00:00,1,2019-02-10 18:00:00,219514.0,73.0,11.0,Sunny,0.0,65.0,6.0,1018.0
3,10626,14433691,R-8293c431,2019-02-10 06:00:00,2019-02-12 00:00:00,1,2019-02-11 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10626,14433691,R-8293c431,2019-02-10 06:00:00,2019-02-12 00:00:00,1,2019-02-11 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71371,525,33014292,R-082bc641,2019-01-01 06:00:00,2019-01-01 18:00:00,0,2019-01-01 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71372,2886,15242565,R-038f3a7c,2019-01-13 06:00:00,2019-01-14 00:00:00,0,2019-01-13 06:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71373,2886,15242565,R-038f3a7c,2019-01-13 06:00:00,2019-01-14 00:00:00,0,2019-01-13 12:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
71374,2886,15242565,R-038f3a7c,2019-01-13 06:00:00,2019-01-14 00:00:00,0,2019-01-13 18:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [28]:
def custom_mode(x):
    modes = x.mode()
    if not modes.empty:
        return modes.iloc[0]  # Get the most frequent value (mode)
    else:
        return np.nan

In [29]:
truck_schedule_route_weather_df = merged_df.groupby(['id_x', 'truck_id', 'route_id'], as_index=False).agg(
    route_avg_temp=('temp','mean'),
    route_avg_wind_speed=('wind_speed','mean'),
    route_avg_precip=('precip', 'mean'),
    route_avg_humidity=('humidity','mean'),
    route_avg_visibility=('visibility', 'mean'),
    route_avg_pressure=('pressure','mean'),
    route_description=('description', custom_mode)
)

In [30]:
truck_schedule_route_weather_df[truck_schedule_route_weather_df['route_avg_temp'].isna()].head()

,id_x,truck_id,route_id,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,route_avg_pressure,route_description
346,347,21093078,R-266863d8,NaN,NaN,NaN,NaN,NaN,NaN,NaN
386,387,18653108,R-c15e9809,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1477,1478,22112064,R-1a08406e,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1479,1480,13779072,R-1d00410b,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1512,1513,16815307,R-3cc18593,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
truck_schedule_route_weather_df = truck_schedule_route_weather_df.rename(columns={'id_x':'id'}) 

In [32]:
schedule_weather_merge_df=truck_schedule_df.merge(truck_schedule_route_weather_df,on=['id','truck_id','route_id'],how='left')

In [34]:
truck_schedule_df

,id,truck_id,route_id,departure_date,estimated_arrival,delay
0,10626,14433691,R-8293c431,2019-02-10 07:00:00,2019-02-11 18:13:12,1
1,7780,21784492,R-2e17fb9b,2019-02-06 07:00:00,2019-02-07 03:19:12,0
2,3953,21680317,R-e4c16974,2019-01-19 07:00:00,2019-01-19 07:42:36,0
3,10264,52804295,R-ed86380e,2019-01-31 07:00:00,2019-02-01 18:51:36,1
4,10945,31870298,R-f5caf40b,2019-01-09 07:00:00,2019-01-10 22:21:36,0
...,...,...,...,...,...,...
12303,10077,11244695,R-b3146c56,2019-01-26 07:00:00,2019-01-27 11:36:00,0
12304,1416,64894212,R-91da63c5,2019-01-07 07:00:00,2019-01-08 03:00:36,0
12305,12265,37803062,R-9db6873e,2019-02-06 07:00:00,2019-02-13 14:27:00,0
12306,525,33014292,R-082bc641,2019-01-01 07:00:00,2019-01-01 12:09:00,0


In [33]:
schedule_weather_merge_df

,id,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,route_avg_pressure,route_description
0,10626,14433691,R-8293c431,2019-02-10 07:00:00,2019-02-11 18:13:12,1,65.000000,11.600000,0.000000,63.000000,6.000000,1019.800000,Sunny
1,7780,21784492,R-2e17fb9b,2019-02-06 07:00:00,2019-02-07 03:19:12,0,42.400000,7.200000,0.000000,68.400000,6.000000,1016.200000,Cloudy
2,3953,21680317,R-e4c16974,2019-01-19 07:00:00,2019-01-19 07:42:36,0,66.000000,11.000000,0.000000,76.000000,6.000000,1016.000000,Clear
3,10264,52804295,R-ed86380e,2019-01-31 07:00:00,2019-02-01 18:51:36,1,34.750000,7.750000,0.000000,84.875000,6.000000,1021.250000,Clear
4,10945,31870298,R-f5caf40b,2019-01-09 07:00:00,2019-01-10 22:21:36,0,55.250000,8.250000,0.000000,89.625000,3.375000,1021.000000,Light drizzle
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,10077,11244695,R-b3146c56,2019-01-26 07:00:00,2019-01-27 11:36:00,0,43.000000,8.000000,0.000000,85.500000,6.000000,1009.500000,Clear
12304,1416,64894212,R-91da63c5,2019-01-07 07:00:00,2019-01-08 03:00:36,0,49.000000,12.000000,0.000000,37.500000,6.000000,1000.000000,Sunny
12305,12265,37803062,R-9db6873e,2019-02-06 07:00:00,2019-02-13 14:27:00,0,52.096774,5.419355,0.016129,92.290323,3.806452,1017.322581,Heavy snow
12306,525,33014292,R-082bc641,2019-01-01 07:00:00,2019-01-01 12:09:00,0,73.000000,7.000000,0.000000,47.000000,6.000000,1012.000000,Sunny


## City weather merge

In [74]:
nearest_hour_schedule_df=truck_schedule_df.copy()
nearest_hour_schedule_df['estimated_arrival_nearest_hour']=nearest_hour_schedule_df['estimated_arrival'].dt.round("H")
nearest_hour_schedule_df['departure_date_nearest_hour']=nearest_hour_schedule_df['departure_date'].dt.round("H")
nearest_hour_schedule_route_df=pd.merge(nearest_hour_schedule_df, routes_df, on='route_id', how='left')


In [75]:
nearest_hour_schedule_route_df

,id_x,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,id_y,origin_id,destination_id,distance,average_hours
0,10626,14433691,R-8293c431,2019-02-10 07:00:00,2019-02-11 18:13:12,1,2019-02-11 18:00:00,2019-02-10 07:00:00,626.0,C-c4565ee8,C-56e39a5e,1761.12,35.22
1,7780,21784492,R-2e17fb9b,2019-02-06 07:00:00,2019-02-07 03:19:12,0,2019-02-07 03:00:00,2019-02-06 07:00:00,828.0,C-3dbd8b2e,C-ef47bdcd,1016.07,20.32
2,3953,21680317,R-e4c16974,2019-01-19 07:00:00,2019-01-19 07:42:36,0,2019-01-19 08:00:00,2019-01-19 07:00:00,646.0,C-c4565ee8,C-280b55fb,35.67,0.71
3,10264,52804295,R-ed86380e,2019-01-31 07:00:00,2019-02-01 18:51:36,1,2019-02-01 19:00:00,2019-01-31 07:00:00,321.0,C-c92599e2,C-fc66f0ab,1793.17,35.86
4,10945,31870298,R-f5caf40b,2019-01-09 07:00:00,2019-01-10 22:21:36,0,2019-01-10 22:00:00,2019-01-09 07:00:00,2178.0,C-328bd8d3,C-3dbd8b2e,1968.09,39.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,10077,11244695,R-b3146c56,2019-01-26 07:00:00,2019-01-27 11:36:00,0,2019-01-27 12:00:00,2019-01-26 07:00:00,1322.0,C-419cd14c,C-2bd47dc5,1429.98,28.60
12304,1416,64894212,R-91da63c5,2019-01-07 07:00:00,2019-01-08 03:00:36,0,2019-01-08 03:00:00,2019-01-07 07:00:00,534.0,C-ef47bdcd,C-d80a1e7d,1000.56,20.01
12305,12265,37803062,R-9db6873e,2019-02-06 07:00:00,2019-02-13 14:27:00,0,2019-02-13 14:00:00,2019-02-06 07:00:00,NaN,NaN,NaN,NaN,NaN
12306,525,33014292,R-082bc641,2019-01-01 07:00:00,2019-01-01 12:09:00,0,2019-01-01 12:00:00,2019-01-01 07:00:00,1171.0,C-6df8beaf,C-7212cebe,257.42,5.15


In [76]:
origin_weather_data_df = city_weather_df.copy()
destination_weather_data_df = city_weather_df.copy()

In [77]:
origin_weather_data_df

,id,date_time,city_id,temp,wind_speed,description,precip,humidity,visibility,pressure
0,31378,2019-01-21 09:00:00,C-9bbf5d8a,77,6,Sunny,0.0,25,6,1020
1,23586,2019-01-17 17:00:00,C-f8f01604,77,10,Sunny,0.0,12,6,1013
2,35280,2019-02-14 23:00:00,C-03bb3e48,-6,11,Partly cloudy,0.0,90,6,1024
3,19059,2019-01-13 02:00:00,C-3dbd8b2e,43,8,Clear,0.0,70,6,1022
4,28860,2019-01-08 11:00:00,C-b25a09de,55,7,Patchy rain possible,0.0,71,6,1011
...,...,...,...,...,...,...,...,...,...,...
52069,20609,2019-01-31 16:00:00,C-7212cebe,-4,3,Overcast,0.0,62,5,1029
52070,29779,2019-02-15 18:00:00,C-b25a09de,37,6,Clear,0.0,74,6,1010
52072,19439,2019-01-28 22:00:00,C-3dbd8b2e,39,9,Clear,0.0,59,6,1022
52073,9078,2019-01-11 05:00:00,C-639c5e36,30,4,Patchy heavy snow,0.0,95,4,1028


In [78]:
nearest_hour_schedule_route_origin_weather_df=pd.merge(nearest_hour_schedule_route_df, origin_weather_data_df, left_on=['departure_date_nearest_hour','origin_id'], right_on=['date_time','city_id'], how='left')

In [79]:
nearest_hour_schedule_route_origin_weather_df = nearest_hour_schedule_route_origin_weather_df.drop(columns='id')

In [80]:
origin_destination_weather_merge_df=pd.merge(nearest_hour_schedule_route_origin_weather_df, destination_weather_data_df, left_on=['estimated_arrival_nearest_hour','destination_id'], right_on=['date_time','city_id'], how='left')

In [81]:
origin_destination_weather_merge_df

,id_x,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,id_y,origin_id,...,id,date_time_y,city_id_y,temp_y,wind_speed_y,description_y,precip_y,humidity_y,visibility_y,pressure_y
0,10626,14433691,R-8293c431,2019-02-10 07:00:00,2019-02-11 18:13:12,1,2019-02-11 18:00:00,2019-02-10 07:00:00,626.0,C-c4565ee8,...,2107.0,2019-02-11 18:00:00,C-56e39a5e,79.0,6.0,Clear,0.0,62.0,6.0,1011.0
1,7780,21784492,R-2e17fb9b,2019-02-06 07:00:00,2019-02-07 03:19:12,0,2019-02-07 03:00:00,2019-02-06 07:00:00,828.0,C-3dbd8b2e,...,13036.0,2019-02-07 03:00:00,C-ef47bdcd,3.0,6.0,Patchy moderate snow,0.0,77.0,4.0,1024.0
2,3953,21680317,R-e4c16974,2019-01-19 07:00:00,2019-01-19 07:42:36,0,2019-01-19 08:00:00,2019-01-19 07:00:00,646.0,C-c4565ee8,...,24705.0,2019-01-19 08:00:00,C-280b55fb,28.0,11.0,Blizzard,0.0,94.0,2.0,1017.0
3,10264,52804295,R-ed86380e,2019-01-31 07:00:00,2019-02-01 18:51:36,1,2019-02-01 19:00:00,2019-01-31 07:00:00,321.0,C-c92599e2,...,37172.0,2019-02-01 19:00:00,C-fc66f0ab,10.0,6.0,Cloudy,0.0,87.0,4.0,1025.0
4,10945,31870298,R-f5caf40b,2019-01-09 07:00:00,2019-01-10 22:21:36,0,2019-01-10 22:00:00,2019-01-09 07:00:00,2178.0,C-328bd8d3,...,19007.0,2019-01-10 22:00:00,C-3dbd8b2e,43.0,4.0,Partly cloudy,0.0,82.0,6.0,1018.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,10077,11244695,R-b3146c56,2019-01-26 07:00:00,2019-01-27 11:36:00,0,2019-01-27 12:00:00,2019-01-26 07:00:00,1322.0,C-419cd14c,...,28213.0,2019-01-27 12:00:00,C-2bd47dc5,45.0,11.0,Partly cloudy,0.0,47.0,6.0,1016.0
12304,1416,64894212,R-91da63c5,2019-01-07 07:00:00,2019-01-08 03:00:36,0,2019-01-08 03:00:00,2019-01-07 07:00:00,534.0,C-ef47bdcd,...,5692.0,2019-01-08 03:00:00,C-d80a1e7d,54.0,10.0,Light rain shower,0.0,95.0,6.0,1011.0
12305,12265,37803062,R-9db6873e,2019-02-06 07:00:00,2019-02-13 14:27:00,0,2019-02-13 14:00:00,2019-02-06 07:00:00,NaN,NaN,...,NaN,NaT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12306,525,33014292,R-082bc641,2019-01-01 07:00:00,2019-01-01 12:09:00,0,2019-01-01 12:00:00,2019-01-01 07:00:00,1171.0,C-6df8beaf,...,19885.0,2019-01-01 12:00:00,C-7212cebe,16.0,6.0,Light snow,0.0,83.0,6.0,1032.0


In [82]:
origin_destination_weather_merge_df.isna().sum()

id_x                                 0
truck_id                             0
route_id                             0
departure_date                       0
estimated_arrival                    0
delay                                0
estimated_arrival_nearest_hour       0
departure_date_nearest_hour          0
id_y                               691
origin_id                          691
destination_id                     691
distance                           691
average_hours                      691
date_time_x                       1247
city_id_x                         1247
temp_x                            1247
wind_speed_x                      1247
description_x                     1247
precip_x                          1247
humidity_x                        1247
visibility_x                      1247
pressure_x                        1247
id                                1046
date_time_y                       1046
city_id_y                         1046
temp_y                   

## Traffic table merge

In [83]:
truck_schedule_for_traffic_df = truck_schedule_df.copy()

In [84]:
truck_schedule_for_traffic_df['estimated_arrival_nearest_hour']=truck_schedule_for_traffic_df['estimated_arrival'].dt.round("H")
truck_schedule_for_traffic_df['departure_date_nearest_hour']=truck_schedule_for_traffic_df['departure_date'].dt.round("H")

In [ ]:
hourly_exploded_scheduled_df=(truck_schedule_for_traffic_df.assign(custom_date = [pd.date_range(start, end, freq='H')  
for start, end
in zip(truck_schedule_for_traffic_df['departure_date'], truck_schedule_for_traffic_df['estimated_arrival'])])  
.explode('custom_date', ignore_index = True))  

hourly_exploded_scheduled_df


,id,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,custom_date
0,10626,14433691,R-8293c431,2019-02-10 07:00:00,2019-02-11 18:13:12,1,2019-02-11 18:00:00,2019-02-10 07:00:00,2019-02-10 07:00:00
1,10626,14433691,R-8293c431,2019-02-10 07:00:00,2019-02-11 18:13:12,1,2019-02-11 18:00:00,2019-02-10 07:00:00,2019-02-10 08:00:00
2,10626,14433691,R-8293c431,2019-02-10 07:00:00,2019-02-11 18:13:12,1,2019-02-11 18:00:00,2019-02-10 07:00:00,2019-02-10 09:00:00
3,10626,14433691,R-8293c431,2019-02-10 07:00:00,2019-02-11 18:13:12,1,2019-02-11 18:00:00,2019-02-10 07:00:00,2019-02-10 10:00:00
4,10626,14433691,R-8293c431,2019-02-10 07:00:00,2019-02-11 18:13:12,1,2019-02-11 18:00:00,2019-02-10 07:00:00,2019-02-10 11:00:00
...,...,...,...,...,...,...,...,...,...
311568,2886,15242565,R-038f3a7c,2019-01-13 07:00:00,2019-01-13 22:31:48,0,2019-01-13 23:00:00,2019-01-13 07:00:00,2019-01-13 18:00:00
311569,2886,15242565,R-038f3a7c,2019-01-13 07:00:00,2019-01-13 22:31:48,0,2019-01-13 23:00:00,2019-01-13 07:00:00,2019-01-13 19:00:00
311570,2886,15242565,R-038f3a7c,2019-01-13 07:00:00,2019-01-13 22:31:48,0,2019-01-13 23:00:00,2019-01-13 07:00:00,2019-01-13 20:00:00
311571,2886,15242565,R-038f3a7c,2019-01-13 07:00:00,2019-01-13 22:31:48,0,2019-01-13 23:00:00,2019-01-13 07:00:00,2019-01-13 21:00:00


In [86]:
scheduled_traffic_df=hourly_exploded_scheduled_df.merge(traffic_df,left_on=['route_id','custom_date'], right_on=['route_id','date_time'],how='left')

In [87]:
scheduled_traffic_df

,id_x,truck_id,route_id,departure_date,estimated_arrival,delay,estimated_arrival_nearest_hour,departure_date_nearest_hour,custom_date,id_y,date_time,no_of_vehicles,accident,time_period
0,10626,14433691,R-8293c431,2019-02-10 07:00:00,2019-02-11 18:13:12,1,2019-02-11 18:00:00,2019-02-10 07:00:00,2019-02-10 07:00:00,690968,2019-02-10 07:00:00,1896.0,0,Morning
1,10626,14433691,R-8293c431,2019-02-10 07:00:00,2019-02-11 18:13:12,1,2019-02-11 18:00:00,2019-02-10 07:00:00,2019-02-10 08:00:00,690969,2019-02-10 08:00:00,2557.0,0,Morning
2,10626,14433691,R-8293c431,2019-02-10 07:00:00,2019-02-11 18:13:12,1,2019-02-11 18:00:00,2019-02-10 07:00:00,2019-02-10 09:00:00,690970,2019-02-10 09:00:00,2695.0,0,Late Morning
3,10626,14433691,R-8293c431,2019-02-10 07:00:00,2019-02-11 18:13:12,1,2019-02-11 18:00:00,2019-02-10 07:00:00,2019-02-10 10:00:00,690971,2019-02-10 10:00:00,2290.0,0,Late Morning
4,10626,14433691,R-8293c431,2019-02-10 07:00:00,2019-02-11 18:13:12,1,2019-02-11 18:00:00,2019-02-10 07:00:00,2019-02-10 11:00:00,690972,2019-02-10 11:00:00,2773.0,0,Late Morning
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
311568,2886,15242565,R-038f3a7c,2019-01-13 07:00:00,2019-01-13 22:31:48,0,2019-01-13 23:00:00,2019-01-13 07:00:00,2019-01-13 18:00:00,1019299,2019-01-13 18:00:00,151.0,0,Evening
311569,2886,15242565,R-038f3a7c,2019-01-13 07:00:00,2019-01-13 22:31:48,0,2019-01-13 23:00:00,2019-01-13 07:00:00,2019-01-13 19:00:00,1019300,2019-01-13 19:00:00,649.0,0,Evening
311570,2886,15242565,R-038f3a7c,2019-01-13 07:00:00,2019-01-13 22:31:48,0,2019-01-13 23:00:00,2019-01-13 07:00:00,2019-01-13 20:00:00,1019301,2019-01-13 20:00:00,2343.0,0,Evening
311571,2886,15242565,R-038f3a7c,2019-01-13 07:00:00,2019-01-13 22:31:48,0,2019-01-13 23:00:00,2019-01-13 07:00:00,2019-01-13 21:00:00,1019302,2019-01-13 21:00:00,2094.0,0,Night


In [ ]:
def custom_agg(values):
    if any(values == 1):
        return 1
    else:
        return 0

In [89]:
scheduled_route_traffic_merge_df = scheduled_traffic_df.groupby(['id_x', 'truck_id', 'route_id'], as_index=False).agg(
                           avg_no_of_vehicles=('no_of_vehicles', 'mean'),
                           accident=('accident', custom_agg)
                       )

In [90]:
scheduled_route_traffic_merge_df

,id_x,truck_id,route_id,avg_no_of_vehicles,accident
0,1,30312694,R-b236e347,2240.428571,0
1,2,59856374,R-29ea762e,1837.272727,1
2,3,12602955,R-a3d67783,1966.000000,0
3,4,46619422,R-31ec9310,2225.571429,0
4,5,10140178,R-a07c5dbd,2206.800000,1
...,...,...,...,...,...
12303,12304,31047945,R-1484a7ea,1846.195652,1
12304,12305,14758432,R-927cf900,1837.450000,1
12305,12306,31370619,R-5a83ad98,1804.737113,1
12306,12307,67332883,R-991530bc,1820.306358,1


## Merge different merged tables

In [91]:
origin_destination_weather_traffic_merge=origin_destination_weather_merge_df.merge(scheduled_route_traffic_merge_df,on=['id_x','truck_id','route_id'],how='left')

In [92]:
schedule_weather_merge_df

,id,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,route_avg_visibility,route_avg_pressure,route_description
0,10626,14433691,R-8293c431,2019-02-10 07:00:00,2019-02-11 18:13:12,1,65.000000,11.600000,0.000000,63.000000,6.000000,1019.800000,Sunny
1,7780,21784492,R-2e17fb9b,2019-02-06 07:00:00,2019-02-07 03:19:12,0,42.400000,7.200000,0.000000,68.400000,6.000000,1016.200000,Cloudy
2,3953,21680317,R-e4c16974,2019-01-19 07:00:00,2019-01-19 07:42:36,0,66.000000,11.000000,0.000000,76.000000,6.000000,1016.000000,Clear
3,10264,52804295,R-ed86380e,2019-01-31 07:00:00,2019-02-01 18:51:36,1,34.750000,7.750000,0.000000,84.875000,6.000000,1021.250000,Clear
4,10945,31870298,R-f5caf40b,2019-01-09 07:00:00,2019-01-10 22:21:36,0,55.250000,8.250000,0.000000,89.625000,3.375000,1021.000000,Light drizzle
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,10077,11244695,R-b3146c56,2019-01-26 07:00:00,2019-01-27 11:36:00,0,43.000000,8.000000,0.000000,85.500000,6.000000,1009.500000,Clear
12304,1416,64894212,R-91da63c5,2019-01-07 07:00:00,2019-01-08 03:00:36,0,49.000000,12.000000,0.000000,37.500000,6.000000,1000.000000,Sunny
12305,12265,37803062,R-9db6873e,2019-02-06 07:00:00,2019-02-13 14:27:00,0,52.096774,5.419355,0.016129,92.290323,3.806452,1017.322581,Heavy snow
12306,525,33014292,R-082bc641,2019-01-01 07:00:00,2019-01-01 12:09:00,0,73.000000,7.000000,0.000000,47.000000,6.000000,1012.000000,Sunny


In [93]:
origin_destination_weather_traffic_merge = origin_destination_weather_traffic_merge.drop(columns=['id_y'])

In [94]:
merged_data_weather_traffic=pd.merge(schedule_weather_merge_df, origin_destination_weather_traffic_merge, left_on=['id', 'truck_id', 'route_id', 'departure_date',
'estimated_arrival', 'delay'], right_on=['id_x', 'truck_id', 'route_id', 'departure_date', 'estimated_arrival', 'delay'], how='left')

In [95]:
merged_data_weather_traffic_trucks = pd.merge(merged_data_weather_traffic, trucks_df, on='truck_id', how='left')

In [96]:
merged_data_weather_traffic_trucks = merged_data_weather_traffic_trucks.drop(columns='id')

In [97]:
final_merge = pd.merge(merged_data_weather_traffic_trucks, drivers_df, left_on='truck_id', right_on = 'vehicle_no', how='left')

In [98]:
def has_midnight(start, end):
    return int(start.date() != end.date())

In [99]:
final_merge['is_midnight'] = final_merge.apply(lambda row: has_midnight(row['departure_date'], row['estimated_arrival']), axis=1)

In [100]:
final_merge

,id_x,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,10626,14433691,R-8293c431,2019-02-10 07:00:00,2019-02-11 18:13:12,1,65.000000,11.600000,0.000000,63.000000,...,93c61598-b,Joshua Higgins,male,50.0,1.0,proactive,8.0,14433691.0,58.01,1
1,7780,21784492,R-2e17fb9b,2019-02-06 07:00:00,2019-02-07 03:19:12,0,42.400000,7.200000,0.000000,68.400000,...,89991562-0,Daniel Hayes,male,46.0,13.0,proactive,6.0,21784492.0,61.20,1
2,3953,21680317,R-e4c16974,2019-01-19 07:00:00,2019-01-19 07:42:36,0,66.000000,11.000000,0.000000,76.000000,...,b20652f9-3,Stephen Potter,male,43.0,7.0,proactive,2.0,21680317.0,61.58,0
3,10264,52804295,R-ed86380e,2019-01-31 07:00:00,2019-02-01 18:51:36,1,34.750000,7.750000,0.000000,84.875000,...,8519efdb-6,Brian Lyons,male,51.0,23.0,proactive,9.0,52804295.0,63.35,1
4,10945,31870298,R-f5caf40b,2019-01-09 07:00:00,2019-01-10 22:21:36,0,55.250000,8.250000,0.000000,89.625000,...,f2519b2d-8,Ryan West,male,51.0,10.0,proactive,3.0,31870298.0,57.41,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,10077,11244695,R-b3146c56,2019-01-26 07:00:00,2019-01-27 11:36:00,0,43.000000,8.000000,0.000000,85.500000,...,2a7f338d-3,David Rivera,male,52.0,21.0,proactive,7.0,11244695.0,61.53,1
12304,1416,64894212,R-91da63c5,2019-01-07 07:00:00,2019-01-08 03:00:36,0,49.000000,12.000000,0.000000,37.500000,...,48129db6-8,Juan Harris,male,45.0,7.0,proactive,7.0,64894212.0,60.93,1
12305,12265,37803062,R-9db6873e,2019-02-06 07:00:00,2019-02-13 14:27:00,0,52.096774,5.419355,0.016129,92.290323,...,a8286acf-9,Mark Mays,male,43.0,5.0,conservative,3.0,37803062.0,42.18,1
12306,525,33014292,R-082bc641,2019-01-01 07:00:00,2019-01-01 12:09:00,0,73.000000,7.000000,0.000000,47.000000,...,9a267df3-a,Walter Webb,male,44.0,7.0,proactive,8.0,33014292.0,62.20,0


In [101]:
final_merge.columns

Index(['id_x', 'truck_id', 'route_id', 'departure_date', 'estimated_arrival',
       'delay', 'route_avg_temp', 'route_avg_wind_speed', 'route_avg_precip',
       'route_avg_humidity', 'route_avg_visibility', 'route_avg_pressure',
       'route_description', 'id_x', 'estimated_arrival_nearest_hour',
       'departure_date_nearest_hour', 'origin_id', 'destination_id',
       'distance', 'average_hours', 'date_time_x', 'city_id_x', 'temp_x',
       'wind_speed_x', 'description_x', 'precip_x', 'humidity_x',
       'visibility_x', 'pressure_x', 'id_y', 'date_time_y', 'city_id_y',
       'temp_y', 'wind_speed_y', 'description_y', 'precip_y', 'humidity_y',
       'visibility_y', 'pressure_y', 'avg_no_of_vehicles', 'accident',
       'truck_age', 'load_capacity_pounds', 'mileage_mpg', 'fuel_type', 'id',
       'driver_id', 'name', 'gender', 'age', 'experience', 'driving_style',
       'ratings', 'vehicle_no', 'average_speed_mph', 'is_midnight'],
      dtype='object')

In [102]:
final_merge.dropna()

,id_x,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,10626,14433691,R-8293c431,2019-02-10 07:00:00,2019-02-11 18:13:12,1,65.00,11.60,0.0,63.000,...,93c61598-b,Joshua Higgins,male,50.0,1.0,proactive,8.0,14433691.0,58.01,1
1,7780,21784492,R-2e17fb9b,2019-02-06 07:00:00,2019-02-07 03:19:12,0,42.40,7.20,0.0,68.400,...,89991562-0,Daniel Hayes,male,46.0,13.0,proactive,6.0,21784492.0,61.20,1
2,3953,21680317,R-e4c16974,2019-01-19 07:00:00,2019-01-19 07:42:36,0,66.00,11.00,0.0,76.000,...,b20652f9-3,Stephen Potter,male,43.0,7.0,proactive,2.0,21680317.0,61.58,0
3,10264,52804295,R-ed86380e,2019-01-31 07:00:00,2019-02-01 18:51:36,1,34.75,7.75,0.0,84.875,...,8519efdb-6,Brian Lyons,male,51.0,23.0,proactive,9.0,52804295.0,63.35,1
4,10945,31870298,R-f5caf40b,2019-01-09 07:00:00,2019-01-10 22:21:36,0,55.25,8.25,0.0,89.625,...,f2519b2d-8,Ryan West,male,51.0,10.0,proactive,3.0,31870298.0,57.41,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12301,5756,27887726,R-127f7d6d,2019-01-28 07:00:00,2019-01-28 18:19:12,0,64.75,8.25,0.0,69.250,...,28ee4db2-3,Robert Short,male,43.0,10.0,conservative,2.0,27887726.0,46.57,0
12302,7539,27887726,R-9d373a0b,2019-02-06 07:00:00,2019-02-07 02:42:36,1,55.60,8.40,0.0,77.200,...,28ee4db2-3,Robert Short,male,43.0,10.0,conservative,2.0,27887726.0,46.57,1
12303,10077,11244695,R-b3146c56,2019-01-26 07:00:00,2019-01-27 11:36:00,0,43.00,8.00,0.0,85.500,...,2a7f338d-3,David Rivera,male,52.0,21.0,proactive,7.0,11244695.0,61.53,1
12304,1416,64894212,R-91da63c5,2019-01-07 07:00:00,2019-01-08 03:00:36,0,49.00,12.00,0.0,37.500,...,48129db6-8,Juan Harris,male,45.0,7.0,proactive,7.0,64894212.0,60.93,1


In [103]:
final_merge = final_merge.loc[:, ~final_merge.columns.duplicated()]

In [104]:
rename_column_names = {
            'id_x': 'unique_id',
            'temp_x': 'origin_temp',
            'wind_speed_x': 'origin_wind_speed',
            'description_x': 'origin_description',
            'precip_x': 'origin_precip',
            'humidity_x': 'origin_humidity',
            'visibility_x': 'origin_visibility',
            'pressure_x': 'origin_pressure',
            'temp_y': 'destination_temp',
            'wind_speed_y': 'destination_wind_speed',
            'description_y': 'destination_description',
            'precip_y': 'destination_precip',
            'humidity_y': 'destination_humidity',
            'visibility_y': 'destination_visibility',
            'pressure_y': 'destination_pressure',
        }
drop_columns = ['date_time_x', 'city_id_x',
                        'id_y', 'date_time_y', 'city_id_y', 'id']

In [105]:
final_merge = final_merge.drop(columns=drop_columns)
final_merge = final_merge.rename(columns= rename_column_names)

In [107]:
final_merge

,unique_id,truck_id,route_id,departure_date,estimated_arrival,delay,route_avg_temp,route_avg_wind_speed,route_avg_precip,route_avg_humidity,...,driver_id,name,gender,age,experience,driving_style,ratings,vehicle_no,average_speed_mph,is_midnight
0,10626,14433691,R-8293c431,2019-02-10 07:00:00,2019-02-11 18:13:12,1,65.000000,11.600000,0.000000,63.000000,...,93c61598-b,Joshua Higgins,male,50.0,1.0,proactive,8.0,14433691.0,58.01,1
1,7780,21784492,R-2e17fb9b,2019-02-06 07:00:00,2019-02-07 03:19:12,0,42.400000,7.200000,0.000000,68.400000,...,89991562-0,Daniel Hayes,male,46.0,13.0,proactive,6.0,21784492.0,61.20,1
2,3953,21680317,R-e4c16974,2019-01-19 07:00:00,2019-01-19 07:42:36,0,66.000000,11.000000,0.000000,76.000000,...,b20652f9-3,Stephen Potter,male,43.0,7.0,proactive,2.0,21680317.0,61.58,0
3,10264,52804295,R-ed86380e,2019-01-31 07:00:00,2019-02-01 18:51:36,1,34.750000,7.750000,0.000000,84.875000,...,8519efdb-6,Brian Lyons,male,51.0,23.0,proactive,9.0,52804295.0,63.35,1
4,10945,31870298,R-f5caf40b,2019-01-09 07:00:00,2019-01-10 22:21:36,0,55.250000,8.250000,0.000000,89.625000,...,f2519b2d-8,Ryan West,male,51.0,10.0,proactive,3.0,31870298.0,57.41,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12303,10077,11244695,R-b3146c56,2019-01-26 07:00:00,2019-01-27 11:36:00,0,43.000000,8.000000,0.000000,85.500000,...,2a7f338d-3,David Rivera,male,52.0,21.0,proactive,7.0,11244695.0,61.53,1
12304,1416,64894212,R-91da63c5,2019-01-07 07:00:00,2019-01-08 03:00:36,0,49.000000,12.000000,0.000000,37.500000,...,48129db6-8,Juan Harris,male,45.0,7.0,proactive,7.0,64894212.0,60.93,1
12305,12265,37803062,R-9db6873e,2019-02-06 07:00:00,2019-02-13 14:27:00,0,52.096774,5.419355,0.016129,92.290323,...,a8286acf-9,Mark Mays,male,43.0,5.0,conservative,3.0,37803062.0,42.18,1
12306,525,33014292,R-082bc641,2019-01-01 07:00:00,2019-01-01 12:09:00,0,73.000000,7.000000,0.000000,47.000000,...,9a267df3-a,Walter Webb,male,44.0,7.0,proactive,8.0,33014292.0,62.20,0
